# Tutorial: Translator Unit Model with Modular Property Package


<img src='translator.PNG' width="300" height="300">

## Learning Outcomes

- Demonstrate use of the Translator unit model in IDAES


## Problem Statement

In this tutorial, we will be transforming an inlet stream that uses the Benzene-Toluene Ideal property package to an outlet stream that uses the Benzene-Toluene Peng-Robinson property package. The stream conditions are as follows:

**Inlet Inputs(Ideal):**

Flow Rate = 100 mol/s

Mole Fraction (benzene) = 0.6

Mole Fraction (toluene) = 0.4

Temperature = 298 K

Pressure = 101325 Pa

**Outlet Results(Peng-Robinson):**

Flow Rate = 100 mol/s

Benzene Molar Concentration = mol/m^3

Toluene Molar Concentration = mol/m^3

Temperature = 298 K

Pressure = 101325 Pa


For more details, please refer to the IDAES documentation: https://idaes-pse.readthedocs.io/en/stable

## Setting up the problem in IDAES

In the following cell, we will be importing the necessary components from Pyomo and IDAES.

In [ ]:
# Import objects from pyomo package 
from pyomo.environ import ConcreteModel, Constraint, value, SolverFactory

# Import the solver
from idaes.core.solvers import get_solver

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import the product unit model
from idaes.models.unit_models import Translator

# Import idaes logger to set output levels
import idaes.logger as idaeslog

# Import the modular property package to create a property block for the flowsheet
from idaes.models.properties.modular_properties.base.generic_property import GenericParameterBlock

# Import the BT_ideal property package to create a configuration file for the GenericParameterBlock based on the ideal eos
from idaes.models.properties.modular_properties.examples.BT_ideal import configuration as configuration_ideal

# Import the BT_PR property package to create a configuration file for the GenericParameterBlock based on the Peng-Robinson eos
from idaes.models.properties.modular_properties.examples.BT_PR import configuration as configuration_PR

# Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
# DOF = Number of Model Variables - Number of Model Constraints
from idaes.core.util.model_statistics import degrees_of_freedom

# Create the ConcreteModel and the FlowsheetBlock objects, and attach the flowsheet block to it.
m = ConcreteModel()

m.fs = FlowsheetBlock(dynamic=False) # dynamic or ss flowsheet needs to be specified here

# Add properties parameter block to the flowsheet with the BT_ideal configuration file
m.fs.properties_ideal = GenericParameterBlock(**configuration_ideal)

# Add properties parameter block to the flowsheet with the BT_PR configuration file
m.fs.properties_PR = GenericParameterBlock(**configuration_PR)

In the following cell, we will be creating the translator unit model, assigning the appropriate property packages to it, and determining the degrees of freedom associated with the translator unit model.

In [ ]:
# Create an instance of the translator unit, attaching it to the flowsheet
# Specify that the property packages to be used for the inlet and outlet streams are the ones we created earlier

m.fs.translator = Translator(
    inlet_property_package=m.fs.properties_ideal,
    outlet_property_package=m.fs.properties_PR,
)


# Call the degrees_of_freedom function, get initial DOF
DOF_initial = degrees_of_freedom(m)
print('The initial degrees of freedom are: {0}'.format(DOF_initial))

In [ ]:
assert DOF_initial == 10

In the following cell, we will be specifying the inlet conditions for the translator block.

In [ ]:
# Fix the inlet conditions

m.fs.translator.inlet.flow_mol.fix(100) # converting to mol/s as unit basis is mol/s
m.fs.translator.inlet.mole_frac_comp[0, "benzene"].fix(0.6)
m.fs.translator.inlet.mole_frac_comp[0, "toluene"].fix(0.4)
m.fs.translator.inlet.pressure.fix(101325) # Pa
m.fs.translator.inlet.temperature.fix(298) # K

In the following cell, we will be adding the translator block constraints to define the relationship between the inlet and outlet property packages and ensuring that there are zero degrees of freedom.

In [ ]:
# Define a translator block to make the constraints more readable
blk = m.fs.translator

# Add translator block constraints
blk.eq_flow_mol = Constraint(
    expr=blk.properties_in[0].flow_mol == blk.properties_out[0].flow_mol
)
blk.eq_benzene_balance = Constraint(
    expr=blk.properties_in[0].flow_mol * blk.properties_in[0].mole_frac_comp["benzene"]
    == blk.properties_out[0].flow_mol_phase_comp["Liq", "benzene"]
)
blk.eq_toluene_balance = Constraint(
    expr=blk.properties_in[0].flow_mol * blk.properties_in[0].mole_frac_comp["toluene"]
    == blk.properties_out[0].flow_mol_phase_comp["Liq", "toluene"]
)
blk.eq_equal_temperature = Constraint(
    expr=blk.properties_in[0].temperature == blk.properties_out[0].temperature
)
blk.eq_equal_pressure = Constraint(
    expr=blk.properties_in[0].pressure == blk.properties_out[0].pressure
)


# Call the degrees_of_freedom function, get final DOF
DOF_final = degrees_of_freedom(m)
print('The final degrees of freedom is: {0}'.format(DOF_final))

In [ ]:
assert DOF_final == 0

### Flowsheet Initialization

In [ ]:
# Initialize the flowsheet, and set the output at WARNING
m.fs.translator.initialize(outlvl=idaeslog.WARNING)

### Obtaining Simulation Results

In [ ]:
# Solve the simulation using the IDAES solver
# Note: If the degrees of freedom = 0, we have a square problem
solver = get_solver()
result = solver.solve(m, tee=True)

In [ ]:
from pyomo.opt import TerminationCondition, SolverStatus

# Check if termination condition is optimal
assert result.solver.termination_condition == TerminationCondition.optimal
assert result.solver.status == SolverStatus.ok

### View Results

In [ ]:
# Display output report
m.fs.translator.report()

# Display inlet and outlet separately
m.fs.translator.inlet.display()
m.fs.translator.outlet.display()


In [ ]:
import pytest

# Check results
assert value(m.fs.translator.inlet.flow_mol[0]) == pytest.approx(100, rel=1e-6)
assert value(m.fs.translator.inlet.mole_frac_comp[0, "benzene"]) == pytest.approx(0.6, rel=1e-6)
assert value(m.fs.translator.inlet.mole_frac_comp[0, "toluene"]) == pytest.approx(0.4, rel=1e-6)

assert value(m.fs.translator.outlet.flow_mol[0]) == pytest.approx(100, rel=1e-6)
# assert value(m.fs.translator.outlet.flow_mol_phase_comp["Liq", "benzene"] == pytest.approx(600, rel=1e-6)
# assert value(m.fs.translator.outlet.flow_mol_phase_comp["Liq", "toluene"] == pytest.approx(400, rel=1e-6)
